In [ ]:
%cd / content/sfm_toaeval/notebooks


In [ ]:
%pip install - -use-feature = in -tree-build ../


In [3]:
import os
import glob
import numpy as np
import subprocess
import logging
import sqlite3
from sfm_toaeval.data.dataset import Dataset
from sfm_toaeval.data.database import Database
from sfm_toaeval.experiment import Experiment
from sfm_toaeval.analysis.analyzer import Analyzer
from sfm_toaeval.visualization.visualizer import Visualizer
from IPython.display import Image
from sqlalchemy import MetaData
from sqlalchemy_schemadisplay import create_schema_graph


In [4]:
raw_path = '../data/raw'
processed_path = '../data/processed/'
databases_path = '../data/databases/'
point_clouds_path = '../data/point_clouds/'
database_path = os.path.join(databases_path, 'main.sqlite')
analysis_database_path = os.path.join(databases_path, 'analysis.sqlite')


In [ ]:
git_path = os.path.join(raw_path, 'SfM_quality_evaluation/')
sequences_path = os.path.join(
    git_path, 'Benchmarking_Camera_Calibration_2008/*/')


In [ ]:
!git clone https://github.com/openMVG/SfM_quality_evaluation.git $git_path
!cp -r -n $sequences_path $processed_path


In [6]:
logging.getLogger().setLevel(logging.INFO)


In [ ]:
K = np.array([[2759.48, 0, 1520.69], [0, 2764.16, 1006.81], [0, 0, 1]])
dataset = Dataset(processed_path, K, 'jpg')
dataset.load()
display(dataset.sequences)


In [ ]:
database = Database(database_path)
dataset.evaluate(
    database,
    databases_path,
    point_clouds_path,
    ['SIFT', 'SURF', 'FAST'],
    ['SIFT', 'SURF', 'DAISY'],
)
database.close()


In [ ]:
graph = create_schema_graph(metadata=MetaData('sqlite:///' + database_path),
                            show_datatypes=True,
                            show_indexes=True,
                            concentrate=False,
                            rankdir='LR',
                            relation_options={
    'fontsize': '6.0',
    'color': 'blue',
    'minlen': 2,
},
)
graph.del_node('sqlite_sequence')
display(Image(graph.create_png()))
graph.write_svg('../reports/figures/ERD.svg')


In [14]:
analyzer = Analyzer(database_path, point_clouds_path, analysis_database_path)
analyzer.analyze()


In [ ]:
%matplotlib agg
visualizer = Visualizer(analysis_database_path)
fig = visualizer.visualize_density_accuracy()
display(fig)
fig.savefig('../reports/figures/radar_chart.svg')


In [ ]:
fig = visualizer.visualize_trade_off()
display(fig)
fig.savefig('../reports/figures/size-error_curves.svg')
